In [1]:
library(tidyverse)
library(repr)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [45]:
student_p <- tempfile () #download zip data from web
download.file("https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip", student_p)
unzip(student_p, "student-por.csv")
unlink(student_p)
por_data <- read.csv("student-por.csv", sep = ";")
write_csv(por_data, "data/por_data.csv")

In [47]:
student_m <- tempfile () #download zip data from web
download.file("https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip", student_m)
unzip(student_m, "student-mat.csv")
unlink(student_m)
mat_data <- read.csv("student-mat.csv", sep = ";")
write_csv(mat_data, "data/mat_data.csv")

In [56]:
tidy_por <- read_csv("data/por_data.csv") %>% select(G3, studytime, schoolsup, internet, absences)
schoolsup <- as.factor("schoolsup")
internet <- as.factor("internet")

Parsed with column specification:
cols(
  .default = col_character(),
  age = col_double(),
  Medu = col_double(),
  Fedu = col_double(),
  traveltime = col_double(),
  studytime = col_double(),
  failures = col_double(),
  famrel = col_double(),
  freetime = col_double(),
  goout = col_double(),
  Dalc = col_double(),
  Walc = col_double(),
  health = col_double(),
  absences = col_double(),
  G1 = col_double(),
  G2 = col_double(),
  G3 = col_double()
)

See spec(...) for full column specifications.



G3,studytime,schoolsup,internet,absences
<dbl>,<dbl>,<chr>,<chr>,<dbl>
11,2,yes,no,4
11,2,no,yes,2
12,2,yes,yes,6
14,3,no,yes,0
13,2,no,no,0
13,2,no,yes,6
13,2,no,yes,0
13,2,yes,no,2
17,2,no,yes,0
